In [ ]:
# 本章的主要内容为编写 Prompt 的原则，在本章中，我们将给出两个编写 Prompt 的原则与一些相关的策略，您可以练习编写
# 高效的 Prompt，从而便捷而有效地使用 LLM。

In [ ]:
# 首先需要安装 OpenAI 库：
# pip install openai

In [1]:
from openai import OpenAI

In [2]:
import os
import openai

In [ ]:
openai.api_key=os.environ.get('INNER_OPENAI_API_KEY')

In [ ]:
openai.base_url = "https://free.v36.cm/v1/"

In [3]:
client = OpenAI()
def get_completion_gpt(prompt, model="gpt-4o-mini", temperature=0): 
    messages = [{"role": "user", "content": prompt}] # 消息队列
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 值越低则输出文本随机性越低
    )
    return response.choices[0].message.content

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}] 
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 值越低则输出文本随机性越低
    )
    return response.choices[0].message.content

In [ ]:
# 两个基本原则
# 原则一：编写清晰、具体的指令
# 您应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并降低您得
# 到无关或不正确响应的可能性。清晰的指令不意味着必须简短，在许多情况下，更长的 Prompt 实际上更清晰，且
# 提供了更多上下文，也就可能产生更详细更相关的输出。
# 使用分隔符清晰地表示输入的不同部分
# 分隔符可以是：```，""，<>，:，<tag> </tag>等。
# 您可以使用任何明显的标点符号将特定的文本部分与 Prompt 的其余部分分开。标记的形式不限，只需要让模型明
# 确知道这是一个单独部分。使用分隔符可以有效避免提示词注入( Prompt injection )。提示词注入是指如果允许
# 用户将某些输入添加到（开发者预定义的） Prompt 中，则所提供的指令可能会与开发者想要执行的操作相冲突，从
# 而使 LLM 遵循用户输入的指令，而非执行开发者预期的操作。即，输入里面可能包含其他指令，会覆盖掉您的指令。
# 对此，使用分隔符是一个不错的策略。

In [4]:
# 在以下的例子中，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符。
# 中文版见下一个 cell
text = """
你应该通过提供尽可能清晰和具体的指令来表达你希望模型做的事情。这样可以引导模型朝着期望的输出方向，\
减少收到无关或错误回答的可能性。不要将写清晰的提示与写简短的提示混淆。在许多情况下，较长的提示提\
供了更多的清晰度和上下文，这可以导致更详细和相关的输出。
"""

In [ ]:
print(text)

In [5]:
prompt = f"""
        将三个反引号之间的文本总结成一句话。
        ```{text}```
    """
response = get_completion_gpt(prompt)
print(response)

提供清晰具体的指令可以有效引导模型生成相关的输出，较长的提示往往能提供更多上下文。


In [ ]:
response_35 = get_completion(prompt)
print(response_35)

In [ ]:
# 寻求结构化的输出
# 输出可以是 Json、HTML 等格式。
# 第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，您可以在 Python 中将其读入字典或列表中。
# 在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [6]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion_gpt(prompt)
print(response)

以下是三本虚构书籍的清单，以 JSON 格式提供：

```json
[
    {
        "book_id": 1,
        "title": "星际迷航者",
        "author": "李明",
        "genre": "科幻"
    },
    {
        "book_id": 2,
        "title": "时间的尽头",
        "author": "张华",
        "genre": "奇幻"
    },
    {
        "book_id": 3,
        "title": "消失的记忆",
        "author": "王芳",
        "genre": "悬疑"
    }
]
```


In [10]:
# 要求模型检查是否满足条件
# 如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指出并停止执行后续
# 的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或错误发生。
# 在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断
# 其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答“未提供步骤”。
# 满足条件的输入（text中提供了步骤）
text_1 = """
泡一杯茶很容易。首先，需要把水烧开。在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
你首先浏览下面<tag></tag>之间的文本,\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
…
第N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
    text:<tag>{text_1}</tag>
"""
response = get_completion_gpt(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。  
第二步 - 拿一个杯子并把茶包放进去。  
第三步 - 将热水倒在茶包上。  
第四步 - 等待一会儿，让茶叶浸泡。  
第五步 - 几分钟后，取出茶包。  
第六步 - 如果愿意，可以加一些糖或牛奶调味。  
第七步 - 享受一杯美味的茶。  


In [11]:
print(prompt)


你首先浏览下面<tag></tag>之间的文本,如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
…
第N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
    text:<tag>
泡一杯茶很容易。首先，需要把水烧开。在等待期间，拿一个杯子并把茶包放进去。一旦水足够热，就把它倒在茶包上。等待一会儿，让茶叶浸泡。几分钟后，取出茶包。如果您愿意，可以加一些糖或牛奶调味。就这样，您可以享受一杯美味的茶了。
</tag>



In [13]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = """
今天阳光明媚，鸟儿在歌唱。这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。人们外出享受着这美好的天气，\
有些人在野餐，有些人在玩游戏或者在草地上放松。这是一个完美\
的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个反引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
…
第N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
  text:```{text_2}```
"""
response = get_completion_gpt(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


In [14]:
# 提供少量示例（少样本提示词，Few-shot prompting）
# 即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。
# 例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和祖父之间的对话的
# 例子。孩子说，“请教我何为耐心”，祖父用下述风格的隐喻来回答。由于我们已经告诉模型要以一致的语气回答，
# 因此现在我们问“请教我何为韧性”，由于模型已经有了这个少样本示例( few-shot example )，它将以类似
# 的语气回答下一个任务。
prompt = """
    您的任务是以一致的风格回答问题。<孩子>: 教我耐心。
    <祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。
    <孩子>: 教我韧性。
    <祖父母>:
"""
response = get_completion_gpt(prompt)
print(response)

在暴风雨中，最坚韧的树木是那些经历过无数风雨的；最美丽的花朵是那些在艰难环境中依然绽放的；最强大的河流是那些在岩石间蜿蜒而过，永不放弃的。


In [ ]:
# 给模型时间去思考
# 如果您发现模型推理过程过于匆忙，导致得出了错误的结论，那么您应该尝试重新构思 Prompt ，要求模型在提
# 供最终答案之前开展思维链，或进行一系列相关推理（a chain or series of relevant reasoning）。换
# 句话说，如果您给模型一个在短时间内或用少量文字无法完成的复杂任务，它的输出结果就容易出错。这种情况对人
# 来说也是类似：如果您要求某人完成复杂的数学问题，又不给足够时间计算出答案，他们也可能会犯错误。因此，在
# 这些情况下，您应该指示模型花更多时间思考问题，让它在任务上花费更多计算资源。

In [15]:
# 指定完成任务所需的步骤
# 接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。
# 首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘
# 要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和人名个数。要求输出
# 以换行符分隔。
text = """
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。虽然略有些摔伤，但他们还是回到
了温馨的家中。尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
prompt_1 = f"""
    执行以下操作：1.用一句话概括下面用三个反引号括起来的文本。2.将摘要翻译成法语。
    3.在法语摘要中列出每个人名。4.输出一个 JSON 对象，其中包含以下键：French_summary，num_names。
    请用换行符分隔您的答案。
    Text:```{text}```
"""
response = get_completion_gpt(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
兄妹杰克和吉尔在爬山打水时不幸摔下山，但他们依然保持冒险精神，继续探索。  
Les frères et sœurs Jack et Jill sont tombés de la montagne en allant chercher de l'eau, mais ils conservent leur esprit d'aventure et continuent à explorer.  
人名：杰克，吉尔  
```json
{
  "French_summary": "Les frères et sœurs Jack et Jill sont tombés de la montagne en allant chercher de l'eau, mais ils conservent leur esprit d'aventure et continuent à explorer.",
  "num_names": 2
}
```


In [ ]:
# 我们将Prompt加以改进，该 Prompt 前半部分不变，同时确切指定了输出的格式。

In [17]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。2-将摘要翻译成法语。
3-在法语摘要中列出每个名称。4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。
请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<法语摘要中的名称列表>
输出 JSON：<带有 French_summary和num_names的 JSON>
Text: <{text}>
"""
response = get_completion_gpt(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：兄妹杰克和吉尔在爬山打水时不幸摔下山，但他们的冒险精神依然高涨，继续探索。

翻译：Les frères et sœurs Jack et Jill sont tombés de la montagne en allant chercher de l'eau, mais leur esprit d'aventure reste intact et ils continuent à explorer.

名称：["Jack", "Jill"]

输出 JSON：{"French_summary":"Les frères et sœurs Jack et Jill sont tombés de la montagne en allant chercher de l'eau, mais leur esprit d'aventure reste intact et ils continuent à explorer.","num_names":2}


In [ ]:
# 指导模型在下结论之前找出一个自己的解法
# 明确地指引模型在匆匆做决策之前，要自己思考出一份解决方案。有时这样会得到更好的结果。这与之前所述思想类似，即给模型时间思考。
# 接下来我们会给出一个问题和一份来自学生的解答，要求模型判断解答是否正确：
prompt = f"""
Determine if the student's solution is correct or not.
Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion_gpt(prompt)
print(response)

In [18]:
prompt = """
    判断学生的解决方案是否正确。
    问题:
    我正在建造一个太阳能发电站，需要帮助计算财务。
        土地费用为 100美元/平方英尺
        我可以以 250美元/平方英尺的价格购买太阳能电池板
        我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
        作为平方英尺数的函数，首年运营的总费用是多少。
    学生的解决方案：
    设x为发电站的大小，单位为平方英尺。
    费用：
        土地费用：100x
        太阳能电池板费用：250x
        维护费用：100,000美元+100x
        总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion_gpt(prompt)
print(response)

学生的解决方案是正确的。我们可以逐步分析一下计算过程：

1. **土地费用**：每平方英尺100美元，发电站的大小为x平方英尺，因此土地费用为 \(100x\) 美元。

2. **太阳能电池板费用**：每平方英尺250美元，发电站的大小为x平方英尺，因此太阳能电池板费用为 \(250x\) 美元。

3. **维护费用**：每年固定维护费用为100,000美元，另外每平方英尺还需支付10美元的维护费用，因此维护费用为 \(100,000 + 10x\) 美元。

4. **总费用**：将以上所有费用相加：
   \[
   \text{总费用} = \text{土地费用} + \text{太阳能电池板费用} + \text{维护费用}
   \]
   \[
   = 100x + 250x + (100,000 + 10x)
   \]
   \[
   = 100x + 250x + 100,000 + 10x
   \]
   \[
   = (100x + 250x + 10x) + 100,000
   \]
   \[
   = 450x + 100,000
   \]

因此，学生的计算结果 \(450x + 100,000\) 美元是正确的。


In [ ]:
# 我:为啥我一提出来，你就能正确推理，之前死说是对的?
# gpt:这是因为模型在回答问题时是基于模式识别和概率推理的，并不是完美地执行逐步数学推理。当你提出问题时，我会立即
# 重新分析问题，并根据你给出的反馈进行修正和改进。
# 在你最初提到“450x + 100,000”时，模型基于先前的计算步骤做出了自信的回答，但模型并没有真正进行细致的加法核对，
# 而是直接依据了过程中的公式进行推理。在没有更多具体的反馈时，模型认为它的回答是正确的。
# 然而，当你指出了计算错误后，我重新仔细审视了每一步，发现了合并同类项时的错误，最终得出了正确的结果。这个过程表明：
# 我可以根据你的反馈进行反思和自我修正。
# 当有外部信息或纠正时，我能够按逻辑修正错误。
# 因此，你的反馈帮助我对问题进行了更精确的分析。模型的“自信”并不意味着它每次都能百分之百正确，尤其是涉及需要细致推理
# 和数学计算的场景时，人工修正是很重要的。

In [ ]:
# 但是注意，学生的解决方案实际上是错误的。（维护费用项100x应为10x，总费用450x应为360x）
# 我们可以通过指导模型先自行找出一个解法来解决这个问题。
# 在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学
# 生的解法是否正确。同时，我们给定了输出的格式要求。通过拆分任务、明确步骤，让模型有更多时间思考，有时可以获
# 得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [19]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：
步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。
使用以下格式：
问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
**学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否**
学生的成绩：正确或不正确
问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000
实际解决方案和步骤：
"""
response = get_completion_gpt(prompt)
print(response)

问题：我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
1. 土地费用：每平方英尺100美元，面积为x平方英尺，所以土地费用为 \(100x\)。
2. 太阳能电池板费用：每平方英尺250美元，面积为x平方英尺，所以太阳能电池板费用为 \(250x\)。
3. 维护费用：固定费用为100,000美元，另外每平方英尺10美元，面积为x平方英尺，所以维护费用为 \(100,000 + 10x\)。

将所有费用相加：
\[
\text{总费用} = \text{土地费用} + \text{太阳能电池板费用} + \text{维护费用}
\]
\[
\text{总费用} = 100x + 250x + (100,000 + 10x) = 100x + 250x + 100,000 + 10x = 360x + 100,000
\]

**学生的计算结果：450x + 100,000**
实际计算结果：360x + 100,000
学生的计算结果和实际计算结果是否相同：否
学生的解决方案和实际解决方案是否相同：否
学生的成绩：不正确


In [ ]:
# 局限性
# 开发大模型相关应用时请务必铭记：
# 虚假知识：模型偶尔会生成一些看似真实实则编造的知识
# 虽然模型在训练过程中接触了大量的知识，但它并没有完全记住所见的信息，因此它不甚清楚自己知识的边界。
# 这意味着它可能会尝试回答主题晦涩难懂的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编
# 造的想法为幻觉（Hallucination）。
# 如下示例展示了大模型的幻觉。我们要求告诉我们 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 产
# 品的信息，事实上，这个公司是真实存在的，但产品是编造的，而模型一本正经地提供了它编造的知识，而且迷惑性很强。

In [20]:
# AeroGlide 这个词可能是品牌或产品的名称
prompt = """
告诉我 Boie 公司生产的 AeroGlide 超薄智能牙刷 的相关信息
"""
response = get_completion_gpt(prompt)
print(response)

Boie 公司生产的 AeroGlide 超薄智能牙刷是一款结合了现代科技与设计的牙刷。以下是一些关于这款牙刷的相关信息：

1. **超薄设计**：AeroGlide 牙刷采用超薄设计，便于携带和使用，适合现代生活的快节奏。

2. **智能功能**：这款牙刷通常配备智能传感器，可以监测刷牙的力度和时间，帮助用户养成更好的刷牙习惯。

3. **高效清洁**：AeroGlide 牙刷的刷头设计旨在提供高效的清洁效果，能够深入牙缝，去除牙菌斑。

4. **可充电**：牙刷通常配备可充电电池，使用方便，减少了更换电池的麻烦。

5. **应用程序支持**：一些型号可能与手机应用程序连接，提供刷牙反馈和个性化的口腔护理建议。

6. **环保材料**：Boie 公司注重环保，可能使用可持续材料制造牙刷，减少对环境的影响。

请注意，具体的功能和特点可能因型号而异，建议查看 Boie 官方网站或相关产品说明以获取最新和最准确的信息。


In [ ]:
# 由于很容易以假乱真，请读者根据在本系列教程中所学知识，在构建自己的应用程序时尽量避免幻觉情况。幻觉是大模型
# 的一个已知缺陷（注：截至2023年7月），OpenAI也在努力解决该问题。
# 在您希望模型根据文本生成回答时，另一种减少幻觉的策略是先要求模型获取来源于该文本的所有引用信息（任何相关引用，
# any relevant quotes），然后要求它基于所引用的信息来回答问题，这使得我们能根据答案追溯源文档，通常对减少幻觉非常有帮助。

In [ ]:
# 关于反斜杠使用的说明：
# 在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（
# newline characters）的影响，但在您调用其他大模型时，需额外考虑换行符是否会影响模型性能。